<a href="https://colab.research.google.com/github/gfx73/PML-DL/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install allennlp
!pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 torchdata==0.4.1 torchtext==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install torchmetrics
!pip install tensorboard

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
     ---------------------------------------- 5.9/5.9 MB 508.5 kB/s eta 0:00:00
  Using cached Markdown-3.4.1-py3-none-any.whl (93 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
     -------------------------------------- 124.6/124.6 kB 1.0 MB/s eta 0:00:00
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
     ---------------------------------------- 3.6/3.6 MB 522.0 kB/s eta 0:00:00
  Using cached Werkzeug-2.2.2-py3-none-any.whl (232 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
     -------------------------------------- 151.7/151.7 kB 3.0 MB/s eta 0:00:00


In [10]:
import sys
from torchtext.datasets import IMDB
from tqdm import tqdm
from torchtext.data.utils import get_tokenizer
import gc
import random
from torch.utils.data import Dataset, DataLoader, Subset
from allennlp.modules.elmo import Elmo, batch_to_ids
import torch
from torch import nn
from torch.nn import functional as F
import torchmetrics
from torch.utils.tensorboard import SummaryWriter

In [3]:
CLASSIFIER_PRETRAINED = False

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  PATH_TO_SAVE_ELMO_CLASSIFIER = '/content/drive/MyDrive/PML&DL/Assignment2/elmo_classifier.pt'
else:
  PATH_TO_SAVE_ELMO_CLASSIFIER = 'elmo_classifier.pt'

In [4]:
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

In [5]:
IMDB_train_iter, IMDB_test_iter = IMDB()

In [6]:
random.seed(11)
tokenizer = get_tokenizer('basic_english')

def get_texts_and_labels(datasplit):
  texts, labels = [], []
  for label, text in tqdm(datasplit):
    texts.append(text)
    labels.append(label=='pos')
  return texts, labels

train_texts, train_labels = get_texts_and_labels(IMDB_train_iter)
test_texts, test_labels = get_texts_and_labels(IMDB_test_iter)

sample_texts_and_labels = lambda tokens, labels: zip(*random.sample(list(zip(tokens, labels)), len(labels)))

train_texts, train_labels = sample_texts_and_labels(train_texts, train_labels)
test_texts, test_labels = sample_texts_and_labels(test_texts, test_labels)


del IMDB_train_iter
del IMDB_test_iter
gc.collect()

25000it [00:00, 34903.67it/s]
25000it [00:00, 36509.80it/s]


0

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

class TokensAndLabels(Dataset):
  def __init__(self, tokens, labels, tokenizer):
    self.tokens = tokens
    self.labels = torch.tensor(labels, dtype=torch.float32)
    self.tokenizer = tokenizer
    self.length = self.labels.shape[0]

  def __getitem__(self, idx):
    return self.tokenizer(self.tokens[idx]), self.labels[idx]

  def __len__(self):
    return self.length


trainset = TokensAndLabels(train_texts, train_labels, tokenizer)
testset = TokensAndLabels(test_texts, test_labels, tokenizer)

valset_size = int(len(testset) * 0.02)
testset_size = len(testset) - valset_size
valset = Subset(testset, range(valset_size))
testset = Subset(testset, range(valset_size, valset_size + testset_size))

class CollateBatch(object):
  def __init__(self, batch_to_ids):
    self.batch_to_ids = batch_to_ids

  def __call__(self, batch):
    tokens_batch, labels_batch = [tokens_and_label[0] for tokens_and_label in batch], [tokens_and_label[1] for tokens_and_label in batch]
    tok_ids = self.batch_to_ids(tokens_batch).to(device)
    labels_batch = torch.tensor(labels_batch, dtype=torch.float32, device=device)
    return tok_ids, labels_batch

collateBatch = CollateBatch(batch_to_ids)

BATCH_SIZE = 32
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collateBatch)
valloader = DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collateBatch)
testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collateBatch)

In [ ]:
class Classifier(nn.Module):
  def __init__(self, embed_size, elmo):
    super(Classifier, self).__init__()
    self.embed_size = embed_size
    self.elmo = elmo
    self.fc1 = nn.Linear(embed_size, 1)
    
  def forward(self, input):
    embs = self.elmo(input)['elmo_representations'][0]
    mean = embs.mean(dim=1)
    x = torch.sigmoid(self.fc1(mean))
    return x


if CLASSIFIER_PRETRAINED:
  classifier = torch.load(PATH_TO_SAVE_ELMO_CLASSIFIER)
else:
  if IN_COLAB:
    options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
    weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"
  else:
    options_file = "options.json"
    weight_file = "weights.hdf5"
  elmo = Elmo(options_file, weight_file, dropout=0, requires_grad=False, num_output_representations=1).to(device)
  classifier = Classifier(1024, elmo=elmo).to(device)

Output()

Output()

In [ ]:
sum(p.numel() for p in classifier.elmo.parameters() if p.requires_grad)

4

In [ ]:
learning_rate = 0.003
epochs = 1
# l2_penalty = 0.001
l2_penalty = 0

optimizer = torch.optim.RMSprop(classifier.parameters(), lr=learning_rate, weight_decay=l2_penalty)
loss_fn = F.binary_cross_entropy_with_logits

In [7]:
def eval_model(model, data, loss_fn):
  acc_metric = torchmetrics.Accuracy().to(device)
  prec_metric = torchmetrics.Precision().to(device)
  rec_metric = torchmetrics.Recall().to(device)
  f1_metric = torchmetrics.F1Score().to(device)
  running_loss = 0
  for x, y in tqdm(data):
    with torch.no_grad():
      y = y.reshape(-1, 1)
      with torch.autocast(device_type=device, dtype=torch.float16):
        preds = model(x)
        loss = loss_fn(preds, y)


      running_loss += loss.item()
      
      y = y.type(torch.int8)
      acc_metric(preds.round(), y)
      prec_metric(preds.round(), y)
      rec_metric(preds.round(), y)
      f1_metric(preds.round(), y)

      # print(y)
      # print(preds.round())
      # print(acc_metric.compute())

  loss = running_loss / len(data)
  acc = acc_metric.compute().item()
  prec = prec_metric.compute().item()
  rec = rec_metric.compute().item()
  f1 = f1_metric.compute().item()
  return loss, acc, prec, rec, f1

In [ ]:
torch.cuda.empty_cache()
train_losses = []
train_accs = []
val_metrics = []
writer = SummaryWriter()

best_val_loss = 1e+8
for epoch in range(epochs):
  running_loss, correct, total = 0, 0, 0
  for iteration, (x_train ,y_train) in tqdm(enumerate(trainloader), total=len(trainloader)):
    optimizer.zero_grad()
    y_train = y_train.reshape(-1,1)
    with torch.autocast(device_type=device, dtype=torch.float16):
      preds = classifier(x_train)
      loss = loss_fn(preds, y_train)

    running_loss += loss.item()
    total += y_train.shape[0]
    correct += preds.round().eq(y_train).sum().item()

    loss.backward()
    optimizer.step()

    if iteration % 50 == 0:
      _loss = running_loss / (iteration + 1)
      acc = correct / total
      print("epoch: {}\titeration: {}\tloss: {}\tthis iteration loss: {}\taccuracy: {}".format(epoch, iteration, _loss, loss, acc))
      writer.add_scalar(f"ElmoClassifier/train/loss/epoch{epoch}/iteration{iteration}", loss, iteration)
      writer.add_scalar(f"ElmoClassifier/train/acc/epoch{epoch}/iteration{iteration}", acc, iteration)

      loss, acc, prec, rec, f1 = eval_model(classifier, valloader, loss_fn)
      writer.add_scalar(f"ElmoClassifier/val/loss/epoch{epoch}/iteration{iteration}", loss, iteration)
      writer.add_scalar(f"ElmoClassifier/val/acc/epoch{epoch}/iteration{iteration}", acc, iteration)
      writer.add_scalar(f"ElmoClassifier/val/prec/epoch{epoch}/iteration{iteration}", prec, iteration)
      writer.add_scalar(f"ElmoClassifier/val/rec/epoch{epoch}/iteration{iteration}", rec, iteration)
      writer.add_scalar(f"ElmoClassifier/val/f1/epoch{epoch}/iteration{iteration}", f1, iteration)


  loss = running_loss / len(trainloader)
  acc = correct / total
  train_losses.append(loss)
  train_accs.append(acc)
  print("epoch {}\ttrain loss : {}\ttrain accuracy : {}".format(epoch, loss, acc))

  loss, acc, prec, rec, f1 = eval_model(classifier, valloader, loss_fn)
  val_metrics.append([loss, acc, prec, rec, f1])
  print("epoch: {}\tval loss: {}\tval acc: {}\tval prec: {}\tval rec: {}\tval f1: {}".format(epoch, loss, acc, prec, rec, f1))
  if best_val_loss > loss:
    torch.save(classifier, PATH_TO_SAVE_ELMO_CLASSIFIER)
    best_val_loss = loss
  if not IN_COLAB:
    torch.save(classifier, f'classifier{epoch}.pt')

  0%|          | 1/782 [00:07<1:32:22,  7.10s/it]

epoch: 0	iteration: 0	loss: 0.6283023357391357	this iteration loss: 0.6283023357391357	accuracy: 0.78125


  7%|▋         | 51/782 [02:48<34:46,  2.85s/it]

epoch: 0	iteration: 50	loss: 0.6828417275466171	this iteration loss: 0.6961194276809692	accuracy: 0.5490196078431373


 13%|█▎        | 101/782 [05:36<38:19,  3.38s/it]

...

epoch: 0	iteration: 750	loss: 0.6216627938372794	this iteration loss: 0.5869169235229492	accuracy: 0.7382656458055925


100%|██████████| 782/782 [44:54<00:00,  3.45s/it]


epoch 0	train loss : 0.6209402933831105	train accuracy : 0.7414


100%|██████████| 16/16 [00:47<00:00,  2.98s/it]


epoch: 0	val loss: 0.5759647786617279	val acc: 0.7940000295639038	val prec: 0.8930232524871826	val rec: 0.7058823704719543	val f1: 0.7885010838508606


In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [11]:
!pip install transformers

In [12]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

109482240

In [14]:
inputs = tokenizer(["Hello world!", 'this is the end'], padding=True, truncation=True, return_tensors="pt")
inputs['input_ids']

tensor([[ 101, 7592, 2088,  999,  102,    0],
        [ 101, 2023, 2003, 1996, 2203,  102]])

In [15]:
outputs = model(**inputs)
outputs.pooler_output

tensor([[-0.8976, -0.3304, -0.7694,  ..., -0.6559, -0.6200,  0.9095],
        [-0.8040, -0.1418,  0.5634,  ...,  0.5303, -0.5168,  0.8233]],
       grad_fn=<TanhBackward0>)

In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"

class TextsAndLabels(Dataset):
  def __init__(self, texts, labels):
    self.texts = texts
    self.labels = torch.tensor(labels, dtype=torch.float32)
    self.length = self.labels.shape[0]

  def __getitem__(self, idx):
    return self.texts[idx], self.labels[idx]

  def __len__(self):
    return self.length


trainset = TextsAndLabels(train_texts, train_labels)
testset = TextsAndLabels(test_texts, test_labels)

valset_size = int(len(testset) * 0.02)
testset_size = len(testset) - valset_size
valset = Subset(testset, range(valset_size))
testset = Subset(testset, range(valset_size, valset_size + testset_size))

class CollateBatch(object):
  def __init__(self, tokenizer, model):
    self.tokenizer = tokenizer
    self.model = model

  def __call__(self, batch):
    texts_batch, labels_batch = [text_and_label[0] for text_and_label in batch], [text_and_label[1] for text_and_label in batch]
    tok_ids = self.tokenizer(texts_batch, padding=True, truncation=True, return_tensors="pt")
    embs = self.model(**tok_ids).pooler_output
    labels_batch = torch.tensor(labels_batch, dtype=torch.float32, device=device)
    return embs, labels_batch

collateBatch = CollateBatch(tokenizer, model)

BATCH_SIZE = 32
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collateBatch)
valloader = DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collateBatch)
testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collateBatch)

In [21]:
class BertClassifier(nn.Module):
  def __init__(self, embed_size=1024):
    super(BertClassifier, self).__init__()
    self.embed_size = embed_size
    self.fc1 = nn.Linear(embed_size, 1)

  def forward(self, embs):
    x = torch.sigmoid(self.fc1(embs))
    return x


classifier = BertClassifier().to(device)

In [23]:
learning_rate = 0.003
epochs = 1
# l2_penalty = 0.001
l2_penalty = 0

optimizer = torch.optim.RMSprop(classifier.parameters(), lr=learning_rate, weight_decay=l2_penalty)
loss_fn = F.binary_cross_entropy_with_logits

In [22]:
torch.cuda.empty_cache()
train_losses = []
train_accs = []
val_metrics = []
writer = SummaryWriter()

best_val_loss = 1e+8
for epoch in range(epochs):
  running_loss, correct, total = 0, 0, 0
  for iteration, (x_train ,y_train) in tqdm(enumerate(trainloader), total=len(trainloader)):
    optimizer.zero_grad()
    y_train = y_train.reshape(-1,1)
    with torch.autocast(device_type=device, dtype=torch.float16):
      preds = classifier(x_train)
      loss = loss_fn(preds, y_train)

    running_loss += loss.item()
    total += y_train.shape[0]
    correct += preds.round().eq(y_train).sum().item()

    loss.backward()
    optimizer.step()

    if iteration % 50 == 0:
      _loss = running_loss / (iteration + 1)
      acc = correct / total
      print("epoch: {}\titeration: {}\tloss: {}\tthis iteration loss: {}\taccuracy: {}".format(epoch, iteration, _loss, loss, acc))
      writer.add_scalar(f"BERTClassifier/train/loss/epoch{epoch}/iteration{iteration}", loss, iteration)
      writer.add_scalar(f"BERTClassifier/train/acc/epoch{epoch}/iteration{iteration}", acc, iteration)

      loss, acc, prec, rec, f1 = eval_model(classifier, valloader, loss_fn)
      writer.add_scalar(f"BERTClassifier/val/loss/epoch{epoch}/iteration{iteration}", loss, iteration)
      writer.add_scalar(f"BERTClassifier/val/acc/epoch{epoch}/iteration{iteration}", acc, iteration)
      writer.add_scalar(f"BERTClassifier/val/prec/epoch{epoch}/iteration{iteration}", prec, iteration)
      writer.add_scalar(f"BERTClassifier/val/rec/epoch{epoch}/iteration{iteration}", rec, iteration)
      writer.add_scalar(f"BERTClassifier/val/f1/epoch{epoch}/iteration{iteration}", f1, iteration)


  loss = running_loss / len(trainloader)
  acc = correct / total
  train_losses.append(loss)
  train_accs.append(acc)
  print("epoch {}\ttrain loss : {}\ttrain accuracy : {}".format(epoch, loss, acc))

  loss, acc, prec, rec, f1 = eval_model(classifier, valloader, loss_fn)
  val_metrics.append([loss, acc, prec, rec, f1])
  print("epoch: {}\tval loss: {}\tval acc: {}\tval prec: {}\tval rec: {}\tval f1: {}".format(epoch, loss, acc, prec, rec, f1))

writer.flush()
writer.close()

NameError: name 'epochs' is not defined

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
if IN_COLAB:
  !kill $(ps aux | awk '{print $2}')